In [22]:
from threeML import *
import matplotlib.pyplot as plt

###########
# DATASET #
###########
folder = "0422_crab_only"
crab = OGIPLike("crab",
                observation=f'{folder}/spectra_Crab.fits',
                response=f'{folder}/spectral_response.rmf.fits')

###################
# ACTIVE CHANNELS #
###################
crab.set_active_measurements('20 - 80')

##################
# SPECTRAL MODEL #
##################
spec = Powerlaw()

##############################
# DEFINITION OF POINT SOURCE #
##############################
ps = PointSource('crab',l=0,b=0,spectral_shape=spec)

####################
# MODEL DEFINITION #
####################
ps_model = Model(ps)

####################
# FIXED PARAMETERS #
####################
ps_model.crab.spectrum.main.Powerlaw.piv = 40

#################
# DISPLAY MODEL #
#################
ps_model.display(complete=True)




[WARNING ] Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.
[WARNING ] FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.
[WARNING ] The response is in an extension called MATRIX, which usually means you also need an ancillary file (ARF) which you didn't provide. You should refer to the documentation  of the instrument and make sure you don't need an ARF.
[INFO    ] Auto-probed noise models:
[INFO    ] - observation: gaussian
[INFO    ] - background: None
[INFO    ] Range 20 - 80 translates to channels 0-16


Model summary:
==============

                  N
Point sources     1
Extended sources  0
Particle sources  0

Free parameters (2):
--------------------

                                  value min_value max_value            unit
crab.spectrum.main.Powerlaw.K       1.0       0.0    1000.0  keV-1 s-1 cm-2
crab.spectrum.main.Powerlaw.index -2.01     -10.0      10.0                

Fixed parameters (3):
---------------------

                                value min_value max_value unit
crab.position.l                   0.0       0.0     360.0  deg
crab.position.b                   0.0     -90.0      90.0  deg
crab.spectrum.main.Powerlaw.piv  40.0      None      None  keV

Properties (0):
--------------------

(none)


Linked parameters (0):
----------------------

(none)

Independent variables:
----------------------

(none)

Linked functions (0):
----------------------

(none)

In [23]:
###################
# DATA DEFINITION #
###################
ps_data = DataList(crab)

#####################
# LIKELIHOOD OBJECT #
#####################
ps_jl = JointLikelihood(ps_model, ps_data)

#######
# FIT #
#######
best_fit_parameters_ps, likelihood_values_ps = ps_jl.fit()

####################
# RESTORE BEST FIT #
####################
ps_jl.restore_best_fit()

#######################
# PLOT DATA AND MODEL #
#######################
plt.figure()
display_spectrum_model_counts(ps_jl,step=True)
plt.savefig('Crab_spectrum_SE.pdf')

[INFO    ] set the minimizer to minuit
Best fit values:



,result,unit
parameter,,
crab.spectrum.main.Powerlaw.K,(4.616 +/- 0.011) x 10^-3,1 / (cm2 keV s)
crab.spectrum.main.Powerlaw.index,-2.207 +/- 0.008,



Correlation matrix:



1.00,0.29
0.29,1.00



Values of -log(likelihood) at the minimum:



,-log(likelihood)
crab,407.711766
total,407.711766



Values of statistical measures:



,statistical measures
AIC,820.280676
BIC,821.089960


In [34]:
import numpy as np
import pickle

val = np.array(best_fit_parameters_ps["value"])
err = np.array(best_fit_parameters_ps["error"])
cor = ps_jl.correlation_matrix
cov = cor * err[:, np.newaxis] * err[np.newaxis, :]

with open(f"./{folder}/crab_parameters.pickle", "wb") as f:
    pickle.dump((val, cov),f)